In [1]:
from ekenv import EKGame
import numpy as np

In [2]:
import numpy as np

# Turning the comb function into one that works on vectors:
comb_vec = np.vectorize(np.math.comb)

def append_probabilities(stacks: np.ndarray, total_deck: np.ndarray) -> np.ndarray:
    """
    Uses the hypergeometric distribution to calculate probabilites of cards
    being in certain places.
    """

    num_stacks = len(stacks)
    num_cards = 13
    
    # For each type of card we know of how many we don't know where they are:
    known_total = total_deck - np.sum(stacks[:, 1:], axis=0)

    # We know how many cards are in what stack:
    stack_counts = stacks[:, 0]

    # Arguments for hypergeometric distribution:
    N = np.sum(known_total)
    K = known_total
    n = stack_counts - np.sum(stacks[:, 1:], axis=1)

    # Calculate hypergeometric distribution for each stack-card-combination:
    NminK = np.tile(N - K, (num_stacks, 1))
    nmink = np.tile(n, (num_cards, 1)).T
    NminKchoosenmink = comb_vec(NminK, nmink)
    Nchoosen = np.tile(comb_vec(N, n), (num_cards, 1)).T
    answ = 1 - NminKchoosenmink / Nchoosen

    # Setting everything we do know to 1:
    answ = np.clip(answ  + (stacks[:, 1:] > 0), 0, 1)

    # Append probabilities to stack (containing factual info) and return:
    return np.append(stacks, answ, axis=1)

In [4]:
g = EKGame()
g.reset(5)
[_, _, cards, _, _] = g.update_and_get_state(False)
print(cards)

[[16  4  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 8  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 8  0  1  2  0  2  0  0  1  0  1  1  0  0]
 [ 8  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 8  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 8  0  1  0  0  0  0  0  0  0  0  0  0  0]]


In [9]:
print(np.round(append_probabilities(cards, g.cards.total_deck)[:, 14:] * 100) / 100)

[[1.   0.3  0.52 0.78 0.67 0.78 0.78 0.78 0.78 0.67 0.67 0.78 0.78]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   1.   0.32 0.55 0.45 0.55 0.55 0.55 0.55 0.45 0.45 0.55 0.55]
 [0.   1.   1.   0.   1.   0.   0.   1.   0.   1.   1.   0.   0.  ]
 [0.   1.   0.32 0.55 0.45 0.55 0.55 0.55 0.55 0.45 0.45 0.55 0.55]
 [0.   1.   0.32 0.55 0.45 0.55 0.55 0.55 0.55 0.45 0.45 0.55 0.55]
 [0.   1.   0.32 0.55 0.45 0.55 0.55 0.55 0.55 0.45 0.45 0.55 0.55]]


In [21]:
def append_probabilities_alt(stacks: np.ndarray, total_deck: np.ndarray) -> np.ndarray:
    """
    Uses the hypergeometric distribution to calculate probabilites of cards
    being in certain places.
    """

    num_stacks = len(stacks)
    num_cards = 13
    
    # For each type of card we know of how many we don't know where they are:
    known_total = total_deck - np.sum(stacks[:, 1:], axis=0)

    # We know how many cards are in what stack:
    stack_counts = stacks[:, 0]

    # Arguments for hypergeometric distribution:
    N = np.sum(known_total)
    K = known_total
    n = stack_counts - np.sum(stacks[:, 1:], axis=1)

    # Calculate hypergeometric distribution for each stack-card-combination:
    NminK = np.reshape((N - K), [1, -1])
    nmink = np.reshape(n, [-1, 1])
    NminKchoosenmink = comb_vec(NminK, nmink)
    Nchoosen = np.reshape(comb_vec(N, n), [-1, 1])
    answ = 1 - NminKchoosenmink / Nchoosen

    # Setting everything we do know to 1:
    answ = np.clip(answ  + (stacks[:, 1:] > 0), 0, 1)

    # Append probabilities to stack (containing factual info) and return:
    return np.append(stacks, answ, axis=1)

:-)


In [23]:
for _ in range(500):
    players = np.random.randint(2, 6)
    g.reset(players)
    [_, _, cards, _, _] = g.update_and_get_state(False)
    p1 = append_probabilities(cards.copy(), g.cards.total_deck)
    p2 = append_probabilities_alt(cards.copy(), g.cards.total_deck)
    assert(np.all(p1 == p2))
    print(p2.shape)
    exit()

(5, 27)
(6, 27)
(6, 27)
(4, 27)
(4, 27)
(6, 27)
(5, 27)
(5, 27)
(5, 27)
(6, 27)
(7, 27)
(5, 27)
(7, 27)
(7, 27)
(4, 27)
(5, 27)
(7, 27)
(5, 27)
(6, 27)
(6, 27)
(4, 27)
(5, 27)
(4, 27)
(7, 27)
(7, 27)
(4, 27)
(5, 27)
(6, 27)
(6, 27)
(6, 27)
(7, 27)
(7, 27)
(7, 27)
(4, 27)
(4, 27)
(5, 27)
(6, 27)
(5, 27)
(5, 27)
(5, 27)
(6, 27)
(5, 27)
(4, 27)
(5, 27)
(7, 27)
(6, 27)
(6, 27)
(5, 27)
(5, 27)
(5, 27)
(7, 27)
(6, 27)
(4, 27)
(7, 27)
(5, 27)
(6, 27)
(6, 27)
(7, 27)
(7, 27)
(6, 27)
(7, 27)
(4, 27)
(7, 27)
(6, 27)
(7, 27)
(5, 27)
(4, 27)
(4, 27)
(6, 27)
(7, 27)
(4, 27)
(4, 27)
(5, 27)
(4, 27)
(5, 27)
(6, 27)
(4, 27)
(7, 27)
(7, 27)
(4, 27)
(5, 27)
(5, 27)
(7, 27)
(4, 27)
(7, 27)
(4, 27)
(4, 27)
(5, 27)
(7, 27)
(5, 27)
(4, 27)
(7, 27)
(7, 27)
(6, 27)
(6, 27)
(5, 27)
(5, 27)
(7, 27)
(6, 27)
(6, 27)
(5, 27)
(7, 27)
(5, 27)
(7, 27)
(4, 27)
(6, 27)
(7, 27)
(6, 27)
(6, 27)
(5, 27)
(7, 27)
(5, 27)
(7, 27)
(6, 27)
(6, 27)
(6, 27)
(6, 27)
(6, 27)
(6, 27)
(6, 27)
(5, 27)
(4, 27)
(4, 27)
(6, 27)
(5, 27)


: 